<a href="https://colab.research.google.com/github/gokulanv/100DaysOfCode_ML/blob/master/Fb_cmu_data_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
def my_etl(df):
    t = pd.DataFrame()

    t['age_group'] = df.D2
    t['male'] = (df.D1 == 1).astype(int)
    
    t['test_last_14_days'] = (df.B10 == 1).astype(int)
    t['test_positive'] = (df.B10a == 1).astype(int)
    t['test_negative'] = (df.B10a == 2).astype(int)
    t['test_unknown'] = (df.B10a == 3).astype(int)

    t['weight'] = df.weight
    return t

In [ ]:
import glob
# Specify your own path
fnames = sorted(glob.glob('extracted_daily_w4/*.csv'), reverse=True)
fname = fnames[0]
print(fname)
df = pd.read_csv(fname, low_memory=False)
%time my_etl(df.iloc[:1_000])

In [ ]:
# parse date out of filenames to be able to record when survey was offered

def my_date(s):
    s1 = s.split('cvid_responses_')[1]
    s2 = s1.split('_recordedby_')[0]
    s3 = s2.replace('_', '-')
    return s3

In [ ]:
%%time

# loop over all data files from wave 4 of survey
# and extract data from most recent file for each day

data = {}
for fname in fnames:
    print(fname)
    data_date = my_date(fname)
    if data_date in data.keys():
        print('already loaded data for', data_date)
    else:
        if data_date <= '2020-09-03':  # earlier waves of survey do not contain key questions for this analysis
            continue
        df_i = pd.read_csv(fname, low_memory=False)
        %time data[data_date] = my_etl(df_i)

In [ ]:
for k in data.keys():
    data[k]['date'] = k

df_all = pd.concat(data.values())
df_all.to_csv('data/fb_data_usa.csv')
print(df_all.shape)